In [1]:
#!/usr/bin/env python
import os
import mne
import argparse

import numpy as np
import matplotlib.pyplot as plt
from mne.viz import iter_topography
from mne.time_frequency import psd_welch
%matplotlib qt

In [2]:
 # Directories
data_dir = 'Z:\Desktop\\'
sub = 1

In [3]:
subj_id = "sub" + str(sub) + "/ball0"

raw_fnames = ["".join([data_dir, subj_id, str(i), "_sss_trans.fif"])
              for i in range(1, 4)]

raws = []
for fname in raw_fnames:
    if os.path.isfile(fname):
        raw = mne.io.Raw(fname, preload=True).crop(tmax=180)
        print(raw.info)
        events = mne.find_events(
            raw, stim_channel="STI101", min_duration=0.003
        )
        raw.pick_types(meg="grad", misc=False)
        raw.notch_filter(50, notch_widths=2)
        raw.filter(l_freq=1.0, h_freq=70)

        raws.append(raw)
        del raw
    else:
        print(fname, "***NOT FOUND")

raw = mne.concatenate_raws(raws)

<ipython-input-3-38e184257d10>:9: RuntimeWarning: This filename (Z:\Desktop\sub1/ball01_sss_trans.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  raw = mne.io.Raw(fname, preload=True).crop(tmax=180)


<Info | 23 non-empty values
 acq_pars: ACQch001 110113 ACQch002 110112 ACQch003 110111 ACQch004 110122 ...
 bads: []
 ch_names: MEG0113, MEG0112, MEG0111, MEG0122, MEG0123, MEG0121, MEG0132, ...
 chs: 204 GRAD, 102 MAG, 12 MISC, 3 STIM
 custom_ref_applied: False
 description: Vectorview system
 dev_head_t: MEG device -> head transform
 dig: 8 items (3 Cardinal, 5 HPI)
 events: 1 item (list)
 experimenter: temporary training acct (meg1)
 file_id: 4 items (dict)
 highpass: 0.1 Hz
 hpi_meas: 1 item (list)
 hpi_results: 1 item (list)
 hpi_subsystem: 2 items (dict)
 line_freq: 50.0
 lowpass: 330.0 Hz
 meas_date: 2019-09-13 15:05:34 UTC
 meas_id: 4 items (dict)
 nchan: 321
 proc_history: 1 item (list)
 proj_id: 1 item (ndarray)
 proj_name: neurofeedback
 projs: []
 sfreq: 1000.0 Hz
 subject_info: 8 items (dict)
>


<ipython-input-3-38e184257d10>:9: RuntimeWarning: This filename (Z:\Desktop\sub1/ball02_sss_trans.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  raw = mne.io.Raw(fname, preload=True).crop(tmax=180)


<Info | 23 non-empty values
 acq_pars: ACQch001 110113 ACQch002 110112 ACQch003 110111 ACQch004 110122 ...
 bads: []
 ch_names: MEG0113, MEG0112, MEG0111, MEG0122, MEG0123, MEG0121, MEG0132, ...
 chs: 204 GRAD, 102 MAG, 12 MISC, 3 STIM
 custom_ref_applied: False
 description: Vectorview system
 dev_head_t: MEG device -> head transform
 dig: 8 items (3 Cardinal, 5 HPI)
 events: 1 item (list)
 experimenter: temporary training acct (meg1)
 file_id: 4 items (dict)
 highpass: 0.1 Hz
 hpi_meas: 1 item (list)
 hpi_results: 1 item (list)
 hpi_subsystem: 2 items (dict)
 line_freq: 50.0
 lowpass: 330.0 Hz
 meas_date: 2019-09-13 15:26:53 UTC
 meas_id: 4 items (dict)
 nchan: 321
 proc_history: 1 item (list)
 proj_id: 1 item (ndarray)
 proj_name: neurofeedback
 projs: []
 sfreq: 1000.0 Hz
 subject_info: 8 items (dict)
>


<ipython-input-3-38e184257d10>:9: RuntimeWarning: This filename (Z:\Desktop\sub1/ball03_sss_trans.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  raw = mne.io.Raw(fname, preload=True).crop(tmax=180)


<Info | 23 non-empty values
 acq_pars: ACQch001 110113 ACQch002 110112 ACQch003 110111 ACQch004 110122 ...
 bads: []
 ch_names: MEG0113, MEG0112, MEG0111, MEG0122, MEG0123, MEG0121, MEG0132, ...
 chs: 204 GRAD, 102 MAG, 12 MISC, 3 STIM
 custom_ref_applied: False
 description: Vectorview system
 dev_head_t: MEG device -> head transform
 dig: 8 items (3 Cardinal, 5 HPI)
 events: 1 item (list)
 experimenter: temporary training acct (meg1)
 file_id: 4 items (dict)
 highpass: 0.1 Hz
 hpi_meas: 1 item (list)
 hpi_results: 1 item (list)
 hpi_subsystem: 2 items (dict)
 line_freq: 50.0
 lowpass: 330.0 Hz
 meas_date: 2019-09-13 15:42:02 UTC
 meas_id: 4 items (dict)
 nchan: 321
 proc_history: 1 item (list)
 proj_id: 1 item (ndarray)
 proj_name: neurofeedback
 projs: []
 sfreq: 1000.0 Hz
 subject_info: 8 items (dict)
>


In [4]:
raw.plot(start=0., duration=20., n_channels=20, events=events)
plt.show()

In [5]:
psds, freqs = mne.time_frequency.psd_welch(raw, n_per_seg=250,
                                           n_overlap=250 / 2,
                                           average='mean', fmin=1.0,
                                           fmax=70)
psds = 20 * np.log10(psds)  # scale to DB

In [7]:
def my_callback(ax, ch_idx):
    """
    This block of code is executed once you click on one of the channel axes
    in the plot. To work with the viz internals, this function should only take
    two parameters, the axis and the channel or data index.
    """

    ax.plot(freqs, psds[ch_idx], color='red')
    ax.set_xlabel('Frequency (Hz)')
    ax.set_ylabel('Power (dB)')
    ax.set(title=ch_idx)

fig = plt.figure(figsize=(12, 8))
for ax, idx in iter_topography(raw.info,
                               fig_facecolor='white',
                               axis_facecolor='white',
                               axis_spinecolor='white',
                               on_pick=my_callback,
                               fig=fig):
    ax.plot(psds[idx], color='red')
    ax.set_title(str(idx))

plt.gcf().suptitle('Power spectral densities')
mne.viz.tight_layout()
plt.show()





name 'ch_idz' is not defined


Traceback (most recent call last):
  File "c:\users\anellim1\develop\thesis\mnematte\.venv\lib\site-packages\matplotlib\cbook\__init__.py", line 196, in process
    func(*args, **kwargs)
  File "c:\users\anellim1\develop\thesis\mnematte\.venv\lib\site-packages\mne\viz\topo.py", line 277, in _plot_topo_onpick
    show_func(ax, ch_idx)
  File "<ipython-input-7-3e1563fc0b3a>", line 11, in my_callback
    ax.set(title=ch_idz)
NameError: name 'ch_idz' is not defined


name 'ch_idz' is not defined


Traceback (most recent call last):
  File "c:\users\anellim1\develop\thesis\mnematte\.venv\lib\site-packages\matplotlib\cbook\__init__.py", line 196, in process
    func(*args, **kwargs)
  File "c:\users\anellim1\develop\thesis\mnematte\.venv\lib\site-packages\mne\viz\topo.py", line 277, in _plot_topo_onpick
    show_func(ax, ch_idx)
  File "<ipython-input-7-3e1563fc0b3a>", line 11, in my_callback
    ax.set(title=ch_idz)
NameError: name 'ch_idz' is not defined


name 'ch_idz' is not defined


...

Traceback (most recent call last):
  File "c:\users\anellim1\develop\thesis\mnematte\.venv\lib\site-packages\matplotlib\cbook\__init__.py", line 196, in process
    func(*args, **kwargs)
  File "c:\users\anellim1\develop\thesis\mnematte\.venv\lib\site-packages\mne\viz\topo.py", line 277, in _plot_topo_onpick
    show_func(ax, ch_idx)
  File "<ipython-input-7-3e1563fc0b3a>", line 11, in my_callback
    ax.set(title=ch_idz)
NameError: name 'ch_idz' is not defined


name 'ch_idz' is not defined


Traceback (most recent call last):
  File "c:\users\anellim1\develop\thesis\mnematte\.venv\lib\site-packages\matplotlib\cbook\__init__.py", line 196, in process
    func(*args, **kwargs)
  File "c:\users\anellim1\develop\thesis\mnematte\.venv\lib\site-packages\mne\viz\topo.py", line 277, in _plot_topo_onpick
    show_func(ax, ch_idx)
  File "<ipython-input-7-3e1563fc0b3a>", line 11, in my_callback
    ax.set(title=ch_idz)
NameError: name 'ch_idz' is not defined


name 'ch_idz' is not defined


Traceback (most recent call last):
  File "c:\users\anellim1\develop\thesis\mnematte\.venv\lib\site-packages\matplotlib\cbook\__init__.py", line 196, in process
    func(*args, **kwargs)
  File "c:\users\anellim1\develop\thesis\mnematte\.venv\lib\site-packages\mne\viz\topo.py", line 277, in _plot_topo_onpick
    show_func(ax, ch_idx)
  File "<ipython-input-7-3e1563fc0b3a>", line 11, in my_callback
    ax.set(title=ch_idz)
NameError: name 'ch_idz' is not defined
